In [1]:
"""
16092022 - 18092022
Бруцкий-Стемпковский

v.01 Частотный анализ текста.

Программа реализует подсчёт предложений избегая сокращения и количество слов в них. Очистка текста от стоп-слов. 
Объединение слов на основе схожести. Схожесть опрееляется на основании коэффициента Танимото и расстояния Левенштейна.
"""

'\n16092022 - 18092022\nБруцкий-Стемпковский\n\nv.01 Частотный анализ текста.\n\nПрограмма реализует подсчёт предложений избегая сокращения и количество слов в них. Очистка текста от стоп-слов. \nОбъединение слов на основе схожести. Схожесть опрееляется на основании коэффициента Танимото и расстояния Левенштейна.\n'

В качестве текста выбрано введение к первому тому "Капитала" К. Маркса. Размер исходного текста превышает 60 000 символов. Написан на русском языке, встречаются слова на латинице. Содержит различные знаки препинания, ссылки в различных скобках, библиотечные шифры, и прочее.

В качестве вспомогательного материала использовал материал из интернета:
1. https://habr.com/ru/post/517410/
2. https://grishaev.me/2012/10/05/1/
3. https://tirinox.ru/levenstein-python/
4. https://pymorphy2.readthedocs.io/en/stable/

Импортируем необходимые библиотеки.

In [2]:
import string
import nltk
import numpy as np
import pandas as pd
import pymorphy2
import time
from functools import lru_cache

Откроем файл, прочтём его содержимое в переменную text, которая будет содержать текст, закроем файл

In [3]:
file_obj = open("text.txt")
text = ""
text = file_obj.read()
file_obj.close()

len(text)

64334

Сперва пройдём весь текст и избавимся от большинства знаков препинания, цифр, скобок, знаков табуляции и новой строки. Но так как нам требуется разделить текст на предложения, не будем убирать знаки окончания предложения (точки, знак вопроса, восклицательный знак). Также не будем пока переводить все буквы в нижний регистр. Планирую использовать тот факт, что при использовании точки для сокращения слова (т. д., проч. и другое) следующее за ним слово пишется с маленькой буквы. Такая точка не считается принаком завершения предложения.

In [4]:
spec_symbol = string.punctuation
spec_symbol = spec_symbol.replace(".", "")
spec_symbol = spec_symbol.replace("!", "")
spec_symbol = spec_symbol.replace("?", "")
spec_symbol += string.digits
spec_symbol += "\n\t\'\"”“–"
symbols_of_end_sentence = ".!?"

spec_symbol

'"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~0123456789\n\t\'"”“–'

Произведем замену всех спецсимволов на знак пробела (обозначается символом \x20).

In [5]:
for symbol in spec_symbol:
    text = text.replace(symbol, "\x20")
text = text.replace("...", "\x20")

len(text)

64332

Пройдем весь текст от начала и до конца в цикле. Если встретим символ конца строки, проверим регистр ближайшего следующего за ним символа. Введем логическую переменную meeting_symbol, которая принимает значение True если мы недавно встретили знак конца предложения, но еще не "решили его судьбу". Будем заменять "правильные" символы конца строки на символ "*" (их там уже нет, хотя можно бы и более специфический символ).

In [6]:
meeting_symbol = False
symbol_of_end_sentence = None

for i in range(len(text)):

    if text[i] in symbols_of_end_sentence:
        meeting_symbol = True
        symbol_of_end_sentence = text[i]
        continue

    if meeting_symbol == True and text[i] != "\x20":
        if text[i] == text[i].upper():
            text = text.replace(symbol_of_end_sentence, "\x20" + "*", 1)
            meeting_symbol = False
            symbol_of_end_sentence = None
        else:
            text = text.replace(symbol_of_end_sentence, "\x20", 1)
            meeting_symbol = False
            symbol_of_end_sentence = None

#выловим последний знак препинания, т.к. за ним ничего нет
for _ in symbols_of_end_sentence:
    text = text.replace(_, "\x20" + "*", 1)

len(text)

64732

Создадим список из строки для подсчёта количество предложений и слов в них. Список бует содержать слова в виде строк и символ "*". Последний будет использован как разделитель.

In [7]:
text_list = text.split()

len(text_list)

8813

Определим количество предложений и слов в них.

In [8]:
number_of_sentence = text_list.count("*")
word_counter = 0
list_of_words_in_sentence = []

for word in text_list:
    if word == "*":
        list_of_words_in_sentence.append(word_counter)
        word_counter = 0
    else:
        word_counter += 1

number_of_sentence, len(list_of_words_in_sentence)
#list_of_words_in_sentence

(400, 400)

Вернемся обрано к строковой переменной. Удалим символы конца предложения, выровняем регистр. 

In [9]:
text = text.replace("*", "")
text = text.lower()

len(text)
#text

64332

Вновь создадим список из слов текста разделяя их по пробелам. В этот раз в списке будут лишь слова без каких-либо знаков препинания. Но у этого списка две пробемы:
1. Содержатся стоп-слова (предлоги, союзы, междометия и проч., не несут смысловой нагрузки.)
2. "Нормальные" слова, несущие смысловую нагрузку, употреблены в различных формах. (Производства, производстве, производящие).

In [10]:
text_list = text.split()

len(text_list)

8413

Удалим все слова, состоящие из 1 - 3 символов. Так мы отсеем большинство предлогов и союзов.

In [11]:
text_list = [_ for _ in text_list if len(_) > 3]

len(text_list)

6000

Импортируем список стоп-слов из библиотеки NLTK.

In [12]:
nltk.download('stopwords')
russian_stopwords = nltk.corpus.stopwords.words("russian")



stop_words = open("stop_words.txt", "w")
for _ in russian_stopwords:
    stop_words.write(_ + "\n")
stop_words.close()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Удалим стоп-слова из нашего списка слов.

In [13]:
text_list = [_ for _ in text_list if not _ in russian_stopwords]

len(text_list)

5455

Реализуем две функции, дающие метрику схожести двух слов. @lru_cache - кэш (память) вызовов функции. Из-за рекурсивного вызова функции самой себя она многократно выполняет свою работу при абсолютно одинаковых входных данных. Преставленный здесь вариант является "средним" по сложности вариантом из [3]. У нас нет необходимости кэшировать только последние вызова функции, т.к. входные данные - уже подготовленные слова, а не тексты 10^6 - 10^7 символов.

In [14]:
def tanimoto(word_a, word_b):
    a, b, c = len(word_a), len(word_a), 0
    for symbol in word_a:
        if symbol in word_b:
            c += 1
    return c/(a + b - c)

def levenstein(a, b):
    @lru_cache(maxsize=len(a) * len(b))
    def recursive(i, j):
        if i == 0 or j == 0:
            return max(i, j)
        elif a[i - 1] == b[j - 1]:
            return recursive(i - 1, j - 1)
        else:
            return 1 + min(
                recursive(i, j - 1),
                recursive(i - 1, j),
                recursive(i - 1, j - 1)
            )
    return recursive(len(a), len(b))

Методом "Подбора" определим критерий схожести слов. При tanimoto >= 0.5 и levenstein <= 4. Слова считаются одинаковыми. Второе в таком случае заменяется на первое.

In [15]:
text_before = open("before.txt", "w")
for _ in text_list:
    text_before.write(_ + "\n")
text_before.close()

С помощью двух вложенных списков сравним каждое слово с каждым... (очень плохая идея =D, работало 3 минуты 20 сек). Выполним замены. Расположим функцию tanimoto первой, т.к. она быстрее рассчитывается.

In [16]:
for i in range(len(text_list)):
    for j in range(i+1, len(text_list)):
        if tanimoto(text_list[i], text_list[j]) >= 0.5 and levenstein(text_list[i], text_list[j]) <= 4:
            text_list[i], text_list[j] = text_list[i], text_list[i]

In [17]:
text_after = open("after.txt", "w")
for _ in text_list:
    text_after.write(_ + "\n")
text_after.close()

Определим множество слов в тексте, подсчитаем количество вхождений каждого слова в текст.

In [18]:
words_set = set(text_list)

len(words_set)

705

In [19]:
dict_of_words = {}
for _ in words_set:
    if text_list.count(_) in dict_of_words:
        dict_of_words[text_list.count(_)].append(_)
    else:
        dict_of_words[text_list.count(_)] = [_]

dict_of_words_file = open("dict_of_words_file.txt", "w")
for _ in sorted(dict_of_words.keys(), reverse = True):
    dict_of_words_file.write(str(_) + "\x20" + "\x20".join(dict_of_words[_]) + "\n")
dict_of_words_file.close()